In [1]:
import os

from dotenv import load_dotenv

In [2]:
OPENAI_API_KEY = os.environ['OPENAI_API_KEY']
PINECONE_API_KEY = os.environ['PINECONE_API_KEY']
PINECONE_ENVIRON = os.environ['PINECONE_ENVIRON']


In [3]:
from langchain.prompts import PromptTemplate
from langchain.prompts import FewShotPromptTemplate
from langchain.prompts.example_selector import SemanticSimilarityExampleSelector
from langchain.vectorstores import Pinecone
from langchain.embeddings import OpenAIEmbeddings

In [4]:
import openai

openai_api_key=OPENAI_API_KEY
model="text-embedding-ada-002"
embed_model=OpenAIEmbeddings(model=model,openai_api_key=openai_api_key)

In [5]:
import pinecone

pinecone.init(
    api_key=PINECONE_API_KEY,
    environment=PINECONE_ENVIRON,
)

index_name="test3"

print(pinecone.describe_index(name=index_name))

f:\GPTeach-By-Example\venv\Lib\site-packages\pinecone\index.py:4: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm


IndexDescription(name='test3', metric='cosine', replicas=1, dimension=1536.0, shards=1, pods=1, pod_type='p1.x1', status={'ready': True, 'state': 'Ready'}, metadata_config=None, source_collection='')


In [6]:
import pandas as pd

# Read the CSV file
df = pd.read_csv('examples.csv')

# Convert DataFrame to dictionary
data_dict = df.to_dict(orient='records')
print(data_dict)


[{'input': 'How do you perform a freeze plug repair?', 'output': '1. Ream hole to original diameter + 0.120 to provide adequate wall thickness.\n2. Fabricate and install freeze plug.'}, {'input': 'How do you install a repair bushing?', 'output': '1. Ream hole to original diameter + 0.120 to provide adequate wall thickness.\n2. Fabricate and install repair bushing.'}, {'input': 'Provide repair for an oversized fastener hole.', 'output': '1. Ream hole to original diameter + 0.120 to provide adequate wall thickness.\n2. Fabricate and install freeze plug.'}, {'input': 'How do you accomplish a doubler repair?', 'output': '1. Trim out damage.\n2. Fabricate and install doubler with 2 fastener rows.'}, {'input': 'How do you repair damage to the fuselage skin?', 'output': '1. Trim out damage.\n2. Fabricate and install doubler with 3 fastener rows.'}, {'input': 'How do you repair a large puncture in a panel?', 'output': '1. Trim out damage.\n2. Fabricate and install doubler with 2 fastener rows.

In [7]:
column_names = df.columns.tolist()
print(column_names[1])


output


In [8]:
# examples = data_dict

# example_prompt = PromptTemplate(
#     input_variables=[column_names[0], column_names[1]],
#     template=f"Input: {{{column_names[0]}}}\nOutput: {{{column_names[1]}}}",
# )
# print(example_prompt)

examples = data_dict

example_prompt = PromptTemplate(
    input_variables=["input", "output"],
    template="Input: {input}\nOutput: {output}",
)
print(example_prompt)



input_variables=['input', 'output'] output_parser=None partial_variables={} template='Input: {input}\nOutput: {output}' template_format='f-string' validate_template=True


In [11]:
example_selector = SemanticSimilarityExampleSelector.from_examples(
    # These are the examples it has available to choose from.
    examples=examples,
    # This is the PromptTemplate being used to format the examples.
    example_prompt=example_prompt,
    # This is the embedding class used to produce embeddings which are used to measure semantic similarity.
    embeddings=embed_model,
    index_name=index_name,
    # This is the VectorStore class that is used to store the embeddings and do a similarity search over.
    vectorstore_cls=Pinecone,
    k=3,
)

print(example_selector)

vectorstore=<langchain.vectorstores.pinecone.Pinecone object at 0x0000028677E99A90> k=3 example_keys=None input_keys=None


In [10]:
dynamic_prompt = FewShotPromptTemplate(
    # We provide an ExampleSelector instead of examples.
    example_selector=example_selector,
    example_prompt=example_prompt,
    prefix="Give step by step repair instructions for every input",
    suffix="Input: {query}\nOutput: ", 
    input_variables=["query"],
) 

print(dynamic_prompt)

input_variables=['query'] output_parser=None partial_variables={} examples=None example_selector=SemanticSimilarityExampleSelector(vectorstore=<langchain.vectorstores.pinecone.Pinecone object at 0x0000028677F0D710>, k=3, example_keys=None, input_keys=None) example_prompt=PromptTemplate(input_variables=['input', 'output'], output_parser=None, partial_variables={}, template='Input: {input}\nOutput: {output}', template_format='f-string', validate_template=True) suffix='Input: {query}\nOutput: ' example_separator='\n\n' prefix='Give step by step repair instructions for every input' template_format='f-string' validate_template=True


In [14]:
# An example with small input, so it selects all examples.
print(dynamic_prompt.format(query="how do you repair a large puncture?"))

Give step by step repair instructions for every input

Input: How do you repair a large puncture in a panel?
Output: 1. Trim out damage.
2. Fabricate and install doubler with 2 fastener rows.

Input: How do you repair a large puncture in a panel?
Output: 1. Trim out damage.
2. Fabricate and install doubler with 2 fastener rows.

Input: How do you repair a large puncture in a panel?
Output: 1. Trim out damage.
2. Fabricate and install doubler with 2 fastener rows.

Input: how do you repair a large puncture?
Output: 


In [ ]:
# from langchain.chat_models import ChatOpenAI

# openai = ChatOpenAI(
#     model="gpt-3.5-turbo",
#     openai_api_key=OPENAI_API_KEY,
#     temperature=0.7
# )

In [12]:
from langchain.llms import OpenAI

openai = OpenAI(
    model_name="text-davinci-003",
    openai_api_key=OPENAI_API_KEY,
    temperature=0.0,
    verbose=True
)

In [13]:
print(openai(dynamic_prompt.format(query="how do you repair an oversized hole?")))

 1. Ream hole to original diameter + 0.120 to provide adequate wall thickness.
2. Fabricate and install freeze plug.


In [ ]:
# You can add an example to an example selector as well.
new_example = {"input": "big", "output": "small"}
dynamic_prompt.example_selector.add_example(new_example)
print(dynamic_prompt.format(adjective="enthusiastic"))